In [122]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import warnings
import folium
from folium.plugins import MeasureControl
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline, set_seed
from geopy.distance import distance

warnings.filterwarnings("ignore")

OUTPUT_DIR = "bert/output"
CACHE_DIR = "bert/cache"
SEED = 42
set_seed(SEED)

In [38]:
model = AutoModelForTokenClassification.from_pretrained(OUTPUT_DIR)
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-GPE_LOC', 'I-GPE_LOC', 'B-PROD', 'I-PROD', 'B-LOC', 'I-LOC', 'B-GPE_ORG', 'I-GPE_ORG', 'B-DRV', 'I-DRV', 'B-EVT', 'I-EVT', 'B-MISC', 'I-MISC']
ner_model = pipeline(
    "ner", model=model, tokenizer=tokenizer, grouped_entities=True
)

In [39]:
text = "Kongens gate er en gate i det sentrale Oslo. Den går fra Akershusstranda parallelt med Kirkegata forbi Bankplassen og krysser Rådhusgata, Tollbugata, Prinsens gate og Karl Johans gate til Stortorvet. Kongens gate fikk sitt navn 1624 av kong Christian IV og er ved siden av Akersgata den eneste gaten som har beholdt det opprinnelige gatenavnet fra byens anleggstid. Kongens gate var en hovedgate i Christian IVs by, antagelig anlagt med omtrent samme trasé som den gamle adkomstveien til Akershus festning nordfra. Den førte til festningen fra Store Voldport i byvollen. Vollporten lå på det nåværende Stortorvet, rett nord for krysset med Karl Johans gate. Gaten er representert i den norske utgaven av Monopol."
text

'Kongens gate er en gate i det sentrale Oslo. Den går fra Akershusstranda parallelt med Kirkegata forbi Bankplassen og krysser Rådhusgata, Tollbugata, Prinsens gate og Karl Johans gate til Stortorvet. Kongens gate fikk sitt navn 1624 av kong Christian IV og er ved siden av Akersgata den eneste gaten som har beholdt det opprinnelige gatenavnet fra byens anleggstid. Kongens gate var en hovedgate i Christian IVs by, antagelig anlagt med omtrent samme trasé som den gamle adkomstveien til Akershus festning nordfra. Den førte til festningen fra Store Voldport i byvollen. Vollporten lå på det nåværende Stortorvet, rett nord for krysset med Karl Johans gate. Gaten er representert i den norske utgaven av Monopol.'

In [40]:
text2 = "Vinter-OL 1994 var de 17. olympiske vinterleker og ble arrangert fra 12. til 27. februar 1994 på Lillehammer, etter at byen ble tildelt lekene under Den internasjonale olympiske komités kongress i Seoul 15. september 1988. Dette var andre gang et vinter-OL ble arrangert i Norge, første gang var i Oslo i 1952. Deler av lekene fant i tillegg til Lillehammer også sted på Hamar (der Vikingskipet og Nordlyshallen ble bygd for nettopp dette formålet), i Hafjell i øyer kommune, i Kvitfjell Alpinanlegg i Ringebu kommune og på Gjøvik (Fjellhallen). Beste nasjon ble Russland med elleve gullmedaljer, mens Norge endte opp med ti gull-, elleve sølv- og fem bronsemedaljer."
text2

'Vinter-OL 1994 var de 17. olympiske vinterleker og ble arrangert fra 12. til 27. februar 1994 på Lillehammer, etter at byen ble tildelt lekene under Den internasjonale olympiske komités kongress i Seoul 15. september 1988. Dette var andre gang et vinter-OL ble arrangert i Norge, første gang var i Oslo i 1952. Deler av lekene fant i tillegg til Lillehammer også sted på Hamar (der Vikingskipet og Nordlyshallen ble bygd for nettopp dette formålet), i Hafjell i øyer kommune, i Kvitfjell Alpinanlegg i Ringebu kommune og på Gjøvik (Fjellhallen). Beste nasjon ble Russland med elleve gullmedaljer, mens Norge endte opp med ti gull-, elleve sølv- og fem bronsemedaljer.'

In [41]:
result = ner_model(text2)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [42]:
def process_ner_results(ner_results):
    output = []
    for token in ner_results:
        entity = int(token['entity_group'].replace("LABEL_", ""))
        output.append({
            "word": token['word'],
            "entity": label_list[entity],
            "score": token['score'],
        })
    res = pd.DataFrame(output)
    
    # Extract location entities
    locs = res[res["entity"].str.contains("LOC")]

    # Merge beginning- with continuation-entities
    locs["continuation"] = np.where(locs["entity"].shift(-1).str.startswith("I-"), locs["word"].shift(-1).str.replace("#", ""), np.nan)
    locs.loc[locs["continuation"].notnull(), "word"] = locs["word"] + locs["continuation"]
    words = locs[~locs["entity"].str.startswith("I-")]["word"].str.lower().values
    
    print("Toponyms:", words)
    toponyms = {}
    uniques, counts = np.unique(words, return_counts=True)
    for i in range(uniques.shape[0]):
        toponyms[uniques[i]] = {"count": counts[i]}
        
    return toponyms


## Gazetteer data

**geonameid**         : integer id of record in geonames database       
**name**              : name of geographical point (utf8) varchar(200)      
**asciiname**         : name of geographical point in plain ascii characters, varchar(200)      
**alternatenames**    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)       
**latitude**          : latitude in decimal degrees (wgs84)     
**longitude**         : longitude in decimal degrees (wgs84)        
**feature class**     : see http://www.geonames.org/export/codes.html, char(1)      
**feature code**      : see http://www.geonames.org/export/codes.html, varchar(10)      
**country code**      : ISO-3166 2-letter country code, 2 characters        
**cc2**               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters        
**admin1 code**       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)        
**admin2 code**       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80)      
**admin3 code**       : code for third level administrative division, varchar(20)       
**admin4 code**       : code for fourth level administrative division, varchar(20)      
**population**        : bigint (8 byte int)         
**elevation**         : in meters, integer      
**dem**               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.     
**timezone**          : the iana timezone id (see file timeZone.txt) varchar(40)        
**modification date** : date of last modification in yyyy-MM-dd format      

In [43]:
def load_gazetteers():
    try:
        gaz_data = pd.read_csv("NO_gazetteers.csv", index_col=0)
    except:
        header = ["geonameid", "name", "asciiname", "alternatenames", "latitude", "longitude", "feature class", "feature code", "country code", "cc2", "admin1 code", "admin2 code", "admin3 code", "admin4 code", "population", "elevation", "dem", "timezone", "modification date"]
        gaz_data = pd.read_csv("NO.txt", sep="\t", names=header)
        tmp = gaz_data[gaz_data["alternatenames"].notnull()][:2]
        tmp["alternatenames"] = tmp["alternatenames"].apply(lambda x: x.split(','))
        alternates = tmp.explode("alternatenames").drop(columns=["name"]).rename(columns={"alternatenames": "name"})

        gaz_data = gaz_data.drop(columns="alternatenames")
        gaz_data = pd.concat([gaz_data, alternates], axis=0).sort_values(["geonameid", "name"]).reset_index(drop=True)
        gaz_data["name"] = gaz_data["name"].str.lower()
        gaz_data.to_csv("NO_gazetteers.csv", index_label="index")
    finally:
        return gaz_data

In [44]:
df_gaz = load_gazetteers()
df_gaz

,geonameid,name,asciiname,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
index,,,,,,,,,,,,,,,,,,
0,470883,jakobselvvand,Jakobselvvatnet,69.54751,30.88881,H,LK,NO,NaN,0.0,NaN,NaN,NaN,0,NaN,116,Europe/Kirov,2020-08-01
1,470883,jakobselvvann,Jakobselvvatnet,69.54751,30.88881,H,LK,NO,NaN,0.0,NaN,NaN,NaN,0,NaN,116,Europe/Kirov,2020-08-01
2,470883,jakobselvvatn,Jakobselvvatnet,69.54751,30.88881,H,LK,NO,NaN,0.0,NaN,NaN,NaN,0,NaN,116,Europe/Kirov,2020-08-01
3,470883,jakobselvvatnet,Jakobselvvatnet,69.54751,30.88881,H,LK,NO,NaN,0.0,NaN,NaN,NaN,0,NaN,116,Europe/Kirov,2020-08-01
4,470883,jakobselvvatnet,Jakobselvvatnet,69.54751,30.88881,H,LK,NO,NaN,0.0,NaN,NaN,NaN,0,NaN,116,Europe/Kirov,2020-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607451,12501428,tinngruveveien,Tinngruveveien,60.12053,10.85292,R,RD,NO,NaN,30.0,3031.0,NaN,NaN,50,214.0,280,Europe/Oslo,2022-12-19
607452,12514346,furre gaard,Furre Gaard,59.25799,5.85087,S,FRM,NO,NaN,14.0,1103.0,NaN,NaN,0,NaN,120,Europe/Oslo,2023-02-24
607453,12514347,furre beach,Furre Beach,59.26050,5.84717,H,BAYS,NO,NaN,14.0,1103.0,NaN,NaN,0,NaN,60,Europe/Oslo,2023-02-24


## Finding the document context

In [156]:
document = process_ner_results(result)

def find_possible_locations(toponyms):
    for toponym in toponyms.copy().keys():
        pos_locs = []
        for item in toponym.split(" "):
            res = df_gaz.loc[df_gaz["name"] == item, ["name", "asciiname", "latitude", "longitude"]].values.tolist()
            pos_locs += res
        if len(pos_locs) > 0:
            toponyms[toponym].update({"possible_locations": pos_locs})
        else:
            del toponyms[toponym]
    return document
        
document = find_possible_locations(document)

pprint(document)

Toponyms: ['lillehammer' 'seoul' 'norge' 'oslo' 'lillehammer' 'hamar' 'vikingskipet'
 'nordlyshallen' 'hafjell' 'øyer kommune' 'kvitfjell alpinanlegg'
 'ringebu kommune' 'gjøvik' 'fjellhallen']
{'gjøvik': {'count': 1,
            'possible_locations': [['gjøvik', 'Gjovik', 60.79574, 10.69155],
                                   ['gjøvik', 'Gjovik', 60.79472, 10.69287],
                                   ['gjøvik', 'Gjovik', 60.79802, 10.68772],
                                   ['gjøvik', 'Gjovik', 60.35496, 10.41732],
                                   ['gjøvik', 'Gjovik', 68.68445, 17.56906]]},
 'hafjell': {'count': 1,
             'possible_locations': [['hafjell', 'Hafjell', 61.22465, 10.48422],
                                    ['hafjell',
                                     'Hafjell',
                                     61.23103,
                                     10.52528]]},
 'hamar': {'count': 1,
           'possible_locations': [['hamar', 'Hamar', 60.7945, 11.06798],
 

#### Define the coordinate grid of Norway

![Norway Grid](grid_crop.jpg)

In [157]:
# Latitudes
LAT_MAX = 72
LAT_MIN = 57

# Longitudes
LON_MAX = 32
LON_MIN = 4

# Precision
DECIMALS = 2
SCALE = 10**DECIMALS

# Calculate offsets
OFFSET_LAT, OFFSET_LON = LAT_MIN*SCALE, LON_MIN*SCALE

# Number of standard deviations used for pruning
NUM_STD=2

In [158]:
def extract_grid_coordinates(grid):
    return (np.argwhere(grid > 0) + [OFFSET_LAT, OFFSET_LON]) / SCALE

#### Helper functions for plotting

In [159]:
def plot_interactive(toponyms, centroid, std_dev, num_std=2):
    m = folium.Map(centroid, zoom_start=5, tiles="Stamen Toner")
    
    m.add_child(MeasureControl())
    
    folium.Circle(
        location=centroid,
        color="gray",
        fill_color="gray",
        radius=(std_dev*num_std*1000),
        metric="metric"
    ).add_to(m)
    
    for item in toponyms.values():
        for loc in item["possible_locations"]:
            lat, lon = loc[-2:]
            folium.CircleMarker(
                    location=[lat, lon],
                    popup=folium.Popup(f"""
                        <ul style="list-style: none">
                        <li><b>Name:</b> {loc[0]}</li>
                        <li><b>ASCII name:</b> {loc[1]}</li>
                        <li><b>Lat:</b> {lat:.2f}</li>
                        <li><b>Lon:</b> {lon:.2f}</li>
                        </ul>
                        """, width=500, max_width=500, height=300),
                    color="blue",
                    fill_color="blue",
                    radius=4
                ).add_to(m)
            
    folium.CircleMarker(
        location=centroid,
        popup=folium.Popup(f"""
            <ul style="list-style: none">
            <li><b>Lat:</b> {centroid[0]:.2f}</li>
            <li><b>Lon:</b> {centroid[1]:.2f}</li>
            </ul>
            """, width=500, max_width=500, height=300),
        color="red",
        fill_color="red",
        radius=6
    ).add_to(m)
    return m

In [160]:
def compute_centroid(toponyms):
    grid = np.zeros(((LAT_MAX-LAT_MIN)*SCALE, (LON_MAX-LON_MIN)*SCALE))
    for item in toponyms.values():
        for location in item["possible_locations"]:
            lat, lon = (np.round(location[-2:], decimals=DECIMALS) * SCALE).astype(int)
            grid[lat - OFFSET_LAT, lon - OFFSET_LON] += item["count"]

    # Calculate centroid
    flat_grid = grid.flatten().astype(int)
    lats = np.repeat(np.arange(LAT_MIN, LAT_MAX, 1/SCALE), grid.shape[1])
    lons = np.tile(np.arange(LON_MIN, LON_MAX, 1/SCALE), grid.shape[0])
    centroid_lat = np.average(lats, weights=flat_grid)
    centroid_lon = np.average(lons, weights=flat_grid)
    centroid = np.array([centroid_lat, centroid_lon])

    # Calculate standard deviation of distances
    coords = extract_grid_coordinates(grid)
    distances = np.array([distance(coord, centroid).km for coord in coords])
    weights = flat_grid[np.argwhere(flat_grid > 0)].reshape(-1)
    std_dev = np.sqrt(np.average(distances**2, weights=weights))

    return grid, std_dev, centroid, distances

### Initial centroid calculation

In [161]:
grid, std_dev, centroid, distances = compute_centroid(document)
print("std_dev:", std_dev)
print("centroid:", centroid)

std_dev: 346.7681334050291
centroid: [61.58365854 11.14682927]


In [162]:
plot_interactive(document, centroid, std_dev, num_std=NUM_STD)

In [163]:
def prune_locations(toponyms, centroid, std_dev, num_std=2, verbose=True):
    # Set cutoff threshold
    threshold = std_dev*num_std
    num_pruned = 0
    for toponym, item in toponyms.items():
        pos_locs, i = item["possible_locations"], 0
        while i < len(pos_locs):
            loc = pos_locs[i]
            loc_coords = loc[-2:]
            dist = distance(loc_coords, centroid).km
                
            # Remove location if distance is greater than threshold
            if dist > threshold:
                num_pruned += 1
                del toponyms[toponym]["possible_locations"][i]
                
            else:
                i+=1
    
    if verbose:
        print(f"Pruned {num_pruned} possible locations")
        
    return toponyms

### Prune locations and recalculate centroid

In [164]:
# Prune possible locations
document = prune_locations(document, centroid, std_dev, num_std=NUM_STD)

# Recalculate centroid
grid, std_dev, centroid, distances = compute_centroid(document)
print("std_dev:", std_dev)
print("centroid:", centroid)

Pruned 8 possible locations
std_dev: 91.40232639648502
centroid: [61.00757576 10.56636364]


In [165]:
plot_interactive(document, centroid, std_dev, num_std=NUM_STD)